In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
pip install selenium webdriver-manager


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
def scrape_blinkit_selenium(search_query):
    """Scrape Blinkit using Selenium (handles JavaScript)."""
    
    # Set up Selenium with headless mode
    options = Options()
    options.add_argument("--headless")  # Run in background
    options.add_argument("--disable-blink-features=AutomationControlled")  # Evade detection
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

    # Initialize WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    try:
        url = f"https://www.blinkit.com/s/?q={search_query}"
        driver.get(url)
        time.sleep(5)  # Allow page to load

        # Extract product details
        products = driver.find_elements(By.CLASS_NAME, "ProductCard__details")
        results = []
        
        for product in products:
            try:
                title = product.find_element(By.CLASS_NAME, "ProductCard__title").text.strip()
                price = product.find_element(By.CLASS_NAME, "ProductCard__price").text.strip()
                results.append({"title": title, "price": price})
            except:
                continue

        return results

    except Exception as e:
        print(f"Error scraping Blinkit: {e}")
        return []
    
    finally:
        driver.quit()

# Example usage
search_query = "AMUL"
blinkit_results = scrape_blinkit_selenium(search_query)
print("Blinkit Results:", blinkit_results)


In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import time

In [ ]:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:112.0) Gecko/20100101 Firefox/112.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36",
]

In [ ]:
def get_random_headers():
    """Generate randomized headers."""
    return {
        "User-Agent": random.choice(USER_AGENTS),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }

In [ ]:
def scrape_blinkit(search_query):
    """Scrape product data from Blinkit."""
    url = f"https://www.blinkit.com/s/?q={search_query}"
    
    for attempt in range(3):  # Retry up to 3 times
        try:
            response = requests.get(url, headers=get_random_headers(), timeout=10)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                products = soup.find_all("div", class_="ProductCard__details")  # Adjust class based on Blinkit's structure
                
                results = []
                for product in products:
                    title = product.find("div", class_="ProductCard__title").text.strip()
                    price = product.find("div", class_="ProductCard__price").text.strip()
                    results.append({"title": title, "price": price})
                
                return results
            
            elif response.status_code == 403:
                print("Access denied! You might be blocked. Retrying with delay...")
            
            elif response.status_code == 429:
                print("Rate limited! Retrying after some delay...")
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching Blinkit: {e}")
        
        time.sleep(random.uniform(5, 10))  # Random delay between 5-10 sec before retrying
    
    print("Failed to fetch data after multiple attempts.")
    return []



In [ ]:
def scrape_zepto(search_query):
    """Scrape product data from Zepto."""
    url = f"https://www.zeptonow.com/search?q={search_query}"
    
    for attempt in range(3):  # Retry up to 3 times
        try:
            response = requests.get(url, headers=get_random_headers(), timeout=10)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                products = soup.find_all("div", class_="ProductCard")  # Adjust class based on Zepto's structure
                
                results = []
                for product in products:
                    title = product.find("div", class_="ProductCard__title").text.strip()
                    price = product.find("div", class_="ProductCard__price").text.strip()
                    results.append({"title": title, "price": price})
                
                return results
            
            elif response.status_code == 403:
                print("Access denied! You might be blocked. Retrying with delay...")
            
            elif response.status_code == 429:
                print("Rate limited! Retrying after some delay...")
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching Zepto: {e}")
        
        time.sleep(random.uniform(5, 10))  # Random delay between 5-10 sec before retrying
    
    print("Failed to fetch data after multiple attempts.")
    return []


In [ ]:
# Example usage
search_query = "milk"
blinkit_results = scrape_blinkit(search_query)
zepto_results = scrape_zepto(search_query)

In [ ]:
print("Blinkit Results:", blinkit_results)
print("Zepto Results:", zepto_results)

In [ ]:
def scrape_blinkit(search_query):
    url = f"https://www.blinkit.com/s/?q={search_query}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("Failed to retrieve Blinkit data")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    products = soup.find_all("div", class_="ProductCard__Details")

    blinkit_data = []
    for product in products:
        name = product.find("div", class_="ProductCard__Name")
        price = product.find("div", class_="ProductCard__Price")
        availability = "In stock" if "Out of stock" not in product.text else "Out of stock"

        if name and price:
            blinkit_data.append({
                "platform": "Blinkit",
                "name": name.text.strip(),
                "price": price.text.strip(),
                "availability": availability
            })

    return blinkit_data


def scrape_zepto(search_query):
    url = f"https://www.zeptonow.com/search?q={search_query}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("Failed to retrieve Zepto data")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    products = soup.find_all("div", class_="ProductItem__Details")

    zepto_data = []
    for product in products:
        name = product.find("div", class_="ProductItem__Name")
        price = product.find("div", class_="ProductItem__Price")
        availability = "In stock" if "Out of stock" not in product.text else "Out of stock"

        if name and price:
            zepto_data.append({
                "platform": "Zepto",
                "name": name.text.strip(),
                "price": price.text.strip(),
                "availability": availability
            })

    return zepto_data
